In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export physics
%nbdev_default_class_level 3

Cells will be exported to geomechy.physics,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
from geomechy.base import BaseElement
from geomechy.fem import bcField
from geomechy.core import NodeSet, ElementSet, GroupSet, ShapeFunctionsManager, MaterialSet, ConstitutiveManager, ElementManager

In [ ]:
from geomechy.base import ItemDict, BaseElement
from geomechy.utils import *
from geomechy.io import jsonReader
from geomechy.materials import *
from geomechy.shape import *
from geomechy.constitutive import *

# Physics
> Main problems

## Hydro Monophasic

In [ ]:
%nbdev_export
class MonoHydroPerm:
    def __init__(self, elements):
        self.elements = elements
        
    def assembleSystem(self):
        pass
    
    def assembleH(self):
        H = np.zeros((self.elements.numNodes,self.elements.numNodes))
        for e in range(self.elements.numElems):
            He = self.getHeMatrix(self.elements[e+1])
            Le = self.elements[e+1].Le
            H = H + np.matmul(Le.T.todense(),np.matmul(He,Le.todense()))
        return H
    
    def getFem(self,data):
        bc = bcField()
        bc.readFromDict(data)
        p, fext = bc.getVectors()
        return p, fext

    
    def getHeMatrix(self, elem):
        He = np.zeros(elem.dN[0].shape[1])
        for g, data in enumerate(elem.gp):
            Ng, Bg, Jg, Wg  = self.getShapeData(elem, g)
            k_perm, v_visco = self.getMaterialData(elem)
            He = He + np.matmul(Bg.T,(k_perm/v_visco)*Bg)*Jg*Wg
        return He

    
    def getShapeData(self, elem, g):
        Wg       = elem.we[g][0]
        Ng       = elem.N[g]
        dNg      = elem.dN[g]
        coords   = elem.coords
        matrixJg = np.matmul(dNg, coords)
        Jg       = np.linalg.det(matrixJg)
        invJg    = np.linalg.inv(matrixJg)
        Bg       = np.matmul(invJg, dNg)
        gCoords  = np.matmul(Ng, coords)
        
        return Ng, Bg, Jg, Wg
 

    def getMaterialData(self, elem):      
        for _, data in elem.mat.items():
            if data.Type == "Soil" or data.Type == "Rock":
                k_perm  = data.permeability
            elif data.Type == "Water" or data.Type == "Oil":
                v_visco = data.dynamic_viscosity
            elif data.Type == "Gas" or data.Type == "Air":
                v_visco = data.dynamic_viscosity
                
        return k_perm, v_visco
        

In [ ]:
jsonFile = "./assets/json/template.json"
data = jsonReader(jsonFile)

nodes = NodeSet()
nodes.readFromDict(data)

elems = ElementSet()
elems.readFromDict(data)

groups = GroupSet(data["Groups"])

shapes = ShapeFunctionsManager(nodes, elems)
shapes.getShapeFunc()

mats = MaterialSet(data["Materials"])

constis = ConstitutiveManager(data["Constitutive"])

elements = ElementManager(nodes, elems, groups, shapes, mats, constis)
elements[1].coords

array([[0.    , 3.6576],
       [0.    , 0.    ],
       [3.048 , 0.3048],
       [3.048 , 3.6576]])

In [ ]:
flux = MonoHydroPerm(elements)
H = flux.assembleH()
p, fext = flux.getFem(data)
p

array([[100.],
       [ -1.],
       [ -1.],
       [100.],
       [  0.],
       [  0.]])

## Geomechanic

In [ ]:
%nbdev_export
class Geomechanic():
    pass

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_constitutive.ipynb.
Converted 03_shape.ipynb.
Converted 04_io.ipynb.
Converted 06_materials.ipynb.
Converted 08_Modules.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
Converted test.ipynb.
